In [1]:
# Hemant Joshi
#1. Compiling a Feature Vector by Using Oriented Fast and RotatedBrief.
#2. Reduce the Dimensions of the Compiled Feature Vector by Using K-Means Clustering
#3. Use K-Nearest Neighbor for Classification of Desired Object 

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.cluster import KMeans

In [3]:
main_folder=r"D:\TYETA\CV\Data_Set"

for folder in os.listdir(main_folder):
    path=os.path.join(main_folder,folder)
    for filename in os.listdir(path):
        p=os.path.join(path,filename)
        a=cv2.imread(p)
        resize=(512,512)
        img=cv2.resize(a,resize)#resize image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        orb = cv2.ORB_create(150)#initialise  detector
        keypoints, descriptors = orb.detectAndCompute(gray, None)
        out=pd.DataFrame(descriptors)
        csv_data=out.to_csv(str(folder)+'exp15.csv', mode='a', header=False)
    print('Extraction of ' + str(folder) +' images Done!')  

Extraction of Cement Road images Done!
Extraction of Kaccha Road images Done!
Extraction of Paver Road images Done!
Extraction of Tarcoal Road images Done!


In [4]:
from csv import writer
from csv import reader

main_folder=r"D:\TYETA\CV\Data_Set"
for folder in os.listdir(main_folder):
    with open(str(folder)+'exp15.csv', 'r') as read_obj, \
        open('wokmeans_exp15.csv', 'a+', newline='') as write_obj:
        csv_reader = reader(read_obj)     # Create a csv.reader object from the input file object
        csv_writer = writer(write_obj)    # Create a csv.writer object from the output file object
        for row in csv_reader:
                csv_writer.writerow(row)      # Add the updated row / list to the output file

In [5]:
#KMeans- Dimension Reduction

In [6]:
import pickle
road_df=pd.read_csv('wokmeans_exp15.csv').iloc[:, 1:]
kmeans = KMeans(n_clusters=5)
model=kmeans.fit(road_df)
print(model)
pickle.dump(model, open('kmeansmodel_exp15.pkl', 'wb')) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [7]:
for folder in os.listdir(main_folder):
    model = pickle.load(open('kmeansmodel_exp15.pkl', "rb"))
    path=os.path.join(main_folder,folder)
    for filename in os.listdir(path):
        p=os.path.join(path,filename)
        a=cv2.imread(p)
        resize=(300,300)
        img=cv2.resize(a,resize)#resize image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        orb = cv2.ORB_create(150)#initialise  detector
        keypoints, descriptors = orb.detectAndCompute(gray, None)
        out=pd.DataFrame(descriptors)
        
        output=pd.DataFrame(model.predict(out))
        hist=np.histogram(output.values,bins=[0,1,2,3,4,5])
        histout=pd.DataFrame(hist[0])
        
        csv_data=histout.transpose().to_csv(str(folder)+'_kmeans_exp15.csv', mode='a', header=False)
        
    print('KMeans of ' + str(folder) +' images Done!')


KMeans of Cement Road images Done!
KMeans of Kaccha Road images Done!
KMeans of Paver Road images Done!
KMeans of Tarcoal Road images Done!


In [8]:
from csv import writer
from csv import reader

main_folder=r"D:\TYETA\CV\Data_Set"
i=1
for folder in os.listdir(main_folder):
    with open(str(folder)+'_kmeans_exp15.csv', 'r') as read_obj, \
        open('FinalORB_exp15.csv', 'a+', newline='') as write_obj:
        csv_reader = reader(read_obj)     # Create a csv.reader object from the input file object
        csv_writer = writer(write_obj)    # Create a csv.writer object from the output file object
        # Read each row of the input csv file as list
        for row in csv_reader:
            row.append(i)            # Append the class in the row / list
            csv_writer.writerow(row)      # Add the updated row / list to the output file
    i+=1

In [9]:
dataset = pd.read_csv('FinalORB_exp15.csv')
X = dataset.iloc[:, 1:6].values
y = dataset.iloc[:, 6].values

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

In [23]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(y_pred)

[3 4 4 2 1 3 2 3 2 3 2 3 2 3 2 1 1 2 2 2 2 1 4 1 4 1 2 4 3 4 2 4 3 2 4 4 1
 3 2 2 2 3 1 4 2 2 2 2 2 1 3 2 2 2 1 4 1 2 3 2 2 1 1 4 1 2 1 3 2 3 1 2 1 3
 2 3 1 4 1 3 1 4 1 2 3 2 2 3 4 4 3 3 3 4 1 2 3 4 3 1 3 1 2 2 3 4 1 4 2 3 1
 4 1 3 2 2 2 4 2 3]


In [24]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4166666666666667
